In [15]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display

# Merge Assessment Information into Registrations 

# Import Data Tables:

In [16]:
registrations = pd.read_csv('content/cleaned_registrations.csv')
student_assessments = pd.read_csv('content/studentAssessment.csv', skiprows=[128223,64073])
assessments_info = pd.read_csv('content/assessments.csv')


print('Registrations')
display(registrations.head())
print('Student Assessments')
display(student_assessments.head())
print('Assessment Info')
display(assessments_info.head())


Registrations


,code_module,code_presentation,id_student,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length
0,AAA,2013J,11391,M,HE Qualification,55<=,0,240,N,Pass,-159.0,268.0,268
1,AAA,2013J,28400,F,HE Qualification,35-55,0,60,N,Pass,-53.0,268.0,268
2,AAA,2013J,30268,F,A Level or Equivalent,35-55,0,60,Y,Withdrawn,-92.0,12.0,268
3,AAA,2013J,31604,F,A Level or Equivalent,35-55,0,60,N,Pass,-52.0,268.0,268
4,AAA,2013J,32885,F,Lower Than A Level,0-35,0,60,N,Pass,-176.0,268.0,268


Student Assessments


,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


Assessment Info


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [17]:
print('Registrations')
display(registrations.info())
print('Student Assessments')
display(student_assessments.info())
print('Assessment Info')
display(assessments_info.info())

Registrations
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32225 entries, 0 to 32224
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 32225 non-null  object 
 1   code_presentation           32225 non-null  object 
 2   id_student                  32225 non-null  int64  
 3   gender                      32225 non-null  object 
 4   highest_education           32225 non-null  object 
 5   age_band                    32225 non-null  object 
 6   num_of_prev_attempts        32225 non-null  int64  
 7   studied_credits             32225 non-null  int64  
 8   disability                  32225 non-null  object 
 9   final_result                32225 non-null  object 
 10  date_registration           32225 non-null  float64
 11  date_unregistration         32225 non-null  float64
 12  module_presentation_length  32225 non-null  int64  
dtypes: float64(2), in

None

Student Assessments
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173910 entries, 0 to 173909
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173910 non-null  int64  
 1   id_student      173910 non-null  int64  
 2   date_submitted  173910 non-null  int64  
 3   is_banked       173910 non-null  int64  
 4   score           173737 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.6 MB


None

Assessment Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    int64  
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.8+ KB


None

# Merge assessment dataframes by assessment

1. Combine `date_submitted` and `date` together to make a column of how long before or after the due date the assessment was submitted.


Save new table


In [24]:
student_assessments[student_assessments['score'].isna()]

,id_assessment,id_student,date_submitted,is_banked,score
215,1752,721259,22,0,NaN
937,1754,260355,127,0,NaN
2364,1760,2606802,180,0,NaN
3358,14984,186780,77,0,NaN
3914,14984,531205,26,0,NaN
...,...,...,...,...,...
148927,34903,582670,241,0,NaN
159249,37415,610738,87,0,NaN
166388,37427,631786,221,0,NaN
169723,37435,648110,62,0,NaN


In [25]:
#merge `studentAssessment` and `assessments` 
assessmentmerge = pd.merge(student_assessments,assessments_info, how = 'left', on='id_assessment')
assessmentmerge = assessmentmerge.sort_values(by = ['id_student','code_presentation','date_submitted'])

#recode assessments as the order of assessments in each presentation
assessmentmerge['assessment_number'] = np.zeros(assessmentmerge.shape[0]).astype(int)
for course in courses['code_module'].unique():
    for present in assessments[assessments['code_module'] == course]['code_presentation'].unique():
        num = 1
        for assess in assessments[(assessments['code_module'] == course) 
            & (assessments['code_presentation'] == present)]['id_assessment']:
            
            assessmentmerge.loc[(assessmentmerge['code_module'] == course) 
            & (assessmentmerge['code_presentation'] == present)
            & (assessmentmerge['id_assessment'] == assess), 'assessment_number'] = num
            num += 1

#create new columns, `past_due`:how long before or after due date the assessment was submitted.]
# and `registration` a combination of `student_id`, `code_module` and `code_presentation` representing the specific
# student registration in a given presentation of a course.
assessmentmerge['past_due'] = (assessmentmerge['date_submitted'] - assessmentmerge['date'])

#drop extraneous columns
#column_order = ['code_module','code_presentation','id_student','']
#assessmentmerge = assessment
#assessmentmerge.to_csv('merged_assessments.csv')
assessmentmerge.head() 


,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,assessment_number,past_due
1756,1758,6516,17,0,60.0,AAA,2014J,TMA,19.0,10.0,1,-2.0
1970,1759,6516,51,0,48.0,AAA,2014J,TMA,54.0,20.0,2,-3.0
2376,1760,6516,116,0,63.0,AAA,2014J,TMA,117.0,20.0,3,-1.0
2685,1761,6516,164,0,61.0,AAA,2014J,TMA,166.0,20.0,4,-2.0
2871,1762,6516,210,0,77.0,AAA,2014J,TMA,215.0,30.0,5,-5.0


# Validate Data
Check for missing data

In [26]:
assessmentmerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173910 entries, 1756 to 45940
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_assessment      173910 non-null  int64  
 1   id_student         173910 non-null  int64  
 2   date_submitted     173910 non-null  int64  
 3   is_banked          173910 non-null  int64  
 4   score              173737 non-null  float64
 5   code_module        173910 non-null  object 
 6   code_presentation  173910 non-null  object 
 7   assessment_type    173910 non-null  object 
 8   date               171046 non-null  float64
 9   weight             173910 non-null  float64
 10  assessment_number  173910 non-null  int32  
 11  past_due           171046 non-null  float64
dtypes: float64(4), int32(1), int64(4), object(3)
memory usage: 16.6+ MB


In [27]:
assessmentmerge[assessmentmerge.score.isna()]

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,assessment_number,past_due
5722,14986,33666,117,0,NaN,BBB,2013B,TMA,89.0,18.0,8,28.0
78481,25350,77241,115,0,NaN,DDD,2013J,TMA,88.0,17.5,3,27.0
117359,34873,126074,20,0,NaN,FFF,2013J,TMA,19.0,12.5,8,1.0
148378,34902,136944,136,0,NaN,FFF,2014J,TMA,136.0,25.0,11,0.0
65175,25334,168243,23,0,NaN,DDD,2013B,TMA,25.0,7.5,8,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
45897,15024,2410544,205,0,NaN,BBB,2014J,TMA,201.0,35.0,5,4.0
119538,34874,2463282,79,0,NaN,FFF,2013J,TMA,47.0,12.5,9,32.0
55406,24292,2500064,113,0,NaN,CCC,2014J,TMA,109.0,22.0,6,4.0
134525,34887,2525633,85,0,NaN,FFF,2014B,TMA,52.0,12.5,9,33.0


There are fewer than 200 assessments missing scores, but we want to have a record for each assessment that a student completed.  How should we fill this missing data?  should they be 0s or median scores?  Let's look at other scores for those students to get some idea of if they tend to get a lot of low score or not.

In [31]:
missing_score_students = assessmentmerge[assessmentmerge.score.isna()]['id_student']
assessmentmerge[assessmentmerge['id_student'].isin(missing_score_students)].head(50)

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,assessment_number,past_due
3290,14984,33666,18,0,58.0,BBB,2013B,TMA,19.0,5.0,6,-1.0
4693,14985,33666,47,0,78.0,BBB,2013B,TMA,47.0,18.0,7,0.0
9479,14991,33666,60,0,100.0,BBB,2013B,CMA,54.0,1.0,1,6.0
5722,14986,33666,117,0,NaN,BBB,2013B,TMA,89.0,18.0,8,28.0
76336,25348,77241,25,0,61.0,DDD,2013J,TMA,25.0,10.0,1,0.0
77015,25349,77241,53,0,81.0,DDD,2013J,TMA,53.0,12.5,2,0.0
78481,25350,77241,115,0,NaN,DDD,2013J,TMA,88.0,17.5,3,27.0
117359,34873,126074,20,0,NaN,FFF,2013J,TMA,19.0,12.5,8,1.0
133188,34886,136944,24,0,82.0,FFF,2014B,TMA,24.0,12.5,8,0.0
137087,34891,136944,486,0,93.0,FFF,2014B,CMA,227.0,0.0,1,259.0


## Transform each assessment for each course presentation into a separate column for that presentation

pivot the table. index = registration, columns = id_assessment, score, weight, past_due

In [ ]:
assessmentmerge.pivot(index = )

# Sort Data
by:
1. student
2. presentation
3. date submitted

pivot:
1. id_student
2. code_module
3. code_presentation
4. assessments in each presentation
    
    a. column per assessment in each code_presentation

# recode assessments as assessment number inside each code_presentation

for each code module

    for each code presentation

        for each assessment

            num = 1

            assessment = num

            num +=1

In [ ]:
assessments.head(30)

In [ ]:
assessments.groupby(['code_module','code_presentation']).head()

In [ ]:
vleinfo = pd.read_csv('/content/vle.csv')
vleinfo.head()

In [ ]:
len(vleinfo.activity_type.value_counts())